In [354]:
import pandas as pd
import requests
import json
import base64

In [355]:
data = pd.read_csv('data/diabetes_prediction_dataset.csv')
inputs = data.iloc[53, :8].to_dict()
inputs

{'gender': 'Female',
 'age': 53.0,
 'hypertension': 0,
 'heart_disease': 0,
 'smoking_history': 'former',
 'bmi': 27.32,
 'HbA1c_level': 7.0,
 'blood_glucose_level': 159}

In [356]:
data.iloc[53, :9]

gender                 Female
age                      53.0
hypertension                0
heart_disease               0
smoking_history        former
bmi                     27.32
HbA1c_level               7.0
blood_glucose_level       159
diabetes                    1
Name: 53, dtype: object

In [ ]:
def prepare_payload(input_data: dict) -> str:
    """
    Convert input data into a JSON payload compatible with TensorFlow Serving.

    Args:
        input_data (dict): Input features as a dictionary.

    Returns:
        str: JSON payload ready for API request.
    """
    features = {}
    for key, value in input_data.items():
        if isinstance(value, float):
            features[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        elif isinstance(value, int):
            features[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        elif isinstance(value, str):
            features[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))
    
    serialized_example = tf.train.Example(
        features=tf.train.Features(feature=features)
    ).SerializeToString()
    
    base64_example = base64.b64encode(serialized_example).decode('utf-8')
    
    payload = {
        "signature_name": "serving_default",
        "instances": [
            {"examples": {"b64": base64_example}}
        ]
    }
    
    return json.dumps(payload)

def send_prediction_request(endpoint: str, payload: str) -> str:
    """
    Send the prepared payload to the TensorFlow Serving model endpoint.

    Args:
        endpoint (str): URL of the API endpoint.
        payload (str): JSON payload.

    Returns:
        str: Prediction result.
    """
    try:
        response = requests.post(endpoint, data=payload, headers={"Content-Type": "application/json"})
        response.raise_for_status()  # Raise an exception for HTTP errors
        predictions = response.json().get("predictions")
        
        if predictions:
            prediction_score = predictions[0][0]
            return "Diabetes" if prediction_score >= 0.5 else "No diabetes"
        else:
            return "Error: No predictions in response."
    except requests.RequestException as e:
        return f"Error during request: {str(e)}"

# Example usage
input_features = {
    "Pregnancies": 2,
    "Glucose": 120.0,
    "BloodPressure": 80,
    "SkinThickness": 25,
    "Insulin": 150.0,
    "BMI": 24.5,
    "DiabetesPedigreeFunction": 0.5,
    "Age": 30
}

# Prepare payload and endpoint
json_payload = prepare_payload(input_features)
api_endpoint = "https://streamlitbaru1-production.up.railway.app/v1/models/diabetes-classification-model:predict"

# Get prediction
result = send_prediction_request(api_endpoint, json_payload)
print(result)

Diabetes
